In [2]:
import tensorflow as tf
import os

In [3]:
# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph 

def freez_graph(model_dir,output_node_names):
    """
    Extract the sub graph defined by the output nodes and convert all its variables into constant
    :model_dir:the root folder containing the checkpoint state file
    :output_node_name:a string containing all the output node's name
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError('Export Directory does not exist. Please specify an export directory: %s'%model_dir )
    if not output_node_names:
        print('You need to supply the name of a node')
        return -1
    #to retreive the checkpoint full path
    checkepoint=tf.train.get_checkpoint_state(model_dir)
    input_checkpoint=checkepoint.model_checkpoint_path
    #print(input_checkpoint)
    
    #to precise the file fullname if the freezed graph
    absolute_model_dir="/".join(input_checkpoint.split('/')[:-1])
    output_graph=absolute_model_dir+'/frozen_model.pb'
    
    #clear device to allow Tensorflow to control on which device it will load operations
    clear_device=True
    with tf.Session(graph=tf.Graph()) as sess:
        #to import meta graph in the current default graph
        saver=tf.train.import_meta_graph(input_checkpoint+'.meta',clear_devices=clear_device)
        
        #to restore the weights
        saver.restore(sess,input_checkpoint)
        
        #using a built-in TF helper to export variables to constant
        output_graph_def=tf.graph_util.convert_variables_to_constants(sess,
                                                                    tf.get_default_graph().as_graph_def(),#the graph_def is used to retrieve the nodes
                                                                    output_node_names.split(',')#the output node names are used to select the usefull nodes
                                                                    )
        #Finaldly to serialize and dump the output graph to the file system
        with tf.gfile.GFile(output_graph,'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph."%len(output_graph_def.node))
    
    return output_graph_def

In [4]:
output_graph_def=freez_graph('../Code/weights/',"input_image,output")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../Code/weights/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
28 ops in the final graph.


In [ ]:
output_graph_def.node